In [1]:
// import { serve } from "jsr:@std/http/server";
import { WebSocketServer, WebSocketClient } from "jsr:@std/ws/mod";
import { kill } from "jsr:@std/node/process";
import { systemMemoryInfo } from "jsr:@std/node/os";
import { BufReader } from "jsr:@std/io/buf_reader";
import { readLines } from "jsr:@std/io/read_lines";

interface ProcessInfo {
  pid: number;
  command: string;
  cpuUsage: number;
  memoryUsage: number;
  status: 'running' | 'stopped' | 'error';
}

class EnhancedProcessManager {
  private processes: Map<number, ProcessInfo> = new Map();
  private processStreams: Map<number, ReadableStream<string>> = new Map();
  private clients: Set<WebSocketClient> = new Set();

  async startProcess(command: string): Promise<number> {
    const process = Deno.run({
      cmd: command.split(' '),
      stdout: 'piped',
      stderr: 'piped',
    });

    const pid = process.pid;
    this.processes.set(pid, {
      pid,
      command,
      cpuUsage: 0,
      memoryUsage: 0,
      status: 'running',
    });

    // Set up stream for process output
    const output = readLines(process.stdout);
    this.processStreams.set(pid, output);

    // Monitor process output
    this.monitorProcessOutput(pid, output);

    console.log(`Started process ${pid}: ${command}`);
    this.broadcastUpdate();
    return pid;
  }

  private async monitorProcessOutput(pid: number, output: ReadableStream<string>): Promise<void> {
    for await (const line of output) {
      console.log(`[Process ${pid}]: ${line}`);
      this.broadcastProcessOutput(pid, line);
    }
    
    // Process has finished
    const process = this.processes.get(pid);
    if (process) {
      process.status = 'stopped';
      this.broadcastUpdate();
    }
  }

  async killProcess(pid: number): Promise<void> {
    if (this.processes.has(pid)) {
      await kill(pid);
      this.processes.get(pid)!.status = 'stopped';
      this.processStreams.delete(pid);
      console.log(`Killed process ${pid}`);
      this.broadcastUpdate();
    } else {
      console.error(`Process ${pid} not found`);
    }
  }

  async sendInput(pid: number, input: string): Promise<void> {
    const process = Deno.run({ cmd: ['kill', '-PIPE', pid.toString()] });
    await process.status();
    
    const encoder = new TextEncoder();
    await Deno.writeAll(Deno.stdin, encoder.encode(input + '\n'));
    console.log(`Sent input to process ${pid}: ${input}`);
  }

  async monitorProcesses(): Promise<void> {
    while (true) {
      for (const [pid, info] of this.processes) {
        if (info.status === 'running') {
          try {
            const status = await Deno.run({ cmd: ['ps', '-p', pid.toString(), '-o', '%cpu,%mem'], stdout: 'piped' }).output();
            const [cpu, mem] = new TextDecoder().decode(status).trim().split('\n')[1].split(/\s+/);
            info.cpuUsage = parseFloat(cpu);
            info.memoryUsage = parseFloat(mem);
          } catch (error) {
            console.error(`Failed to monitor process ${pid}:`, error);
            info.status = 'error';
          }
        }
      }
      this.broadcastUpdate();
      await new Promise(resolve => setTimeout(resolve, 5000));
    }
  }

  async getSystemMemoryInfo(): Promise<Deno.SystemMemoryInfo> {
    return await systemMemoryInfo();
  }

  private broadcastUpdate(): void {
    const update = JSON.stringify({
      type: 'processUpdate',
      processes: Array.from(this.processes.values()),
    });
    for (const client of this.clients) {
      client.send(update);
    }
  }

  private broadcastProcessOutput(pid: number, output: string): void {
    const message = JSON.stringify({
      type: 'processOutput',
      pid,
      output,
    });
    for (const client of this.clients) {
      client.send(message);
    }
  }

  async startWebInterface(port: number): Promise<void> {
    const wss = new WebSocketServer(port);
    console.log(`WebSocket server running on ws://localhost:${port}`);

    for await (const conn of wss) {
      this.handleWebSocketConnection(conn);
    }
  }

  private handleWebSocketConnection(conn: WebSocketClient): void {
    this.clients.add(conn);
    conn.on('message', async (message: string) => {
      const data = JSON.parse(message);
      switch (data.action) {
        case 'startProcess':
          await this.startProcess(data.command);
          break;
        case 'killProcess':
          await this.killProcess(data.pid);
          break;
        case 'sendInput':
          await this.sendInput(data.pid, data.input);
          break;
      }
    });
    conn.on('close', () => this.clients.delete(conn));
    this.broadcastUpdate();
  }
}

// Usage
const manager = new EnhancedProcessManager();

// Start the process monitor
manager.monitorProcesses();

// Start the web interface
manager.startWebInterface(8080);

// You can also interact with the manager programmatically
await manager.startProcess('python long_running_script.py');
await manager.startProcess('node server.js');


import { assertEquals, assertExists, assertArrayIncludes } from "jsr:@std/assert";
import { delay } from "jsr:@std/async/delay";

Deno.test("EnhancedProcessManager", async (t) => {
  await t.step("should start a process", async () => {
    const manager = new EnhancedProcessManager();
    const pid = await manager.startProcess("echo hello");
    assertExists(pid);
    assertEquals(typeof pid, "number");
  });

  await t.step("should list running processes", async () => {
    const manager = new EnhancedProcessManager();
    const pid = await manager.startProcess("sleep 5");
    await delay(100); // Give some time for the process to start
    const processes = Array.from(manager["processes"].values());
    assertArrayIncludes(processes, [{
      pid,
      command: "sleep 5",
      status: "running",
    }], "Started process should be in the list");
  });

  await t.step("should kill a process", async () => {
    const manager = new EnhancedProcessManager();
    const pid = await manager.startProcess("sleep 10");
    await delay(100); // Give some time for the process to start
    await manager.killProcess(pid);
    await delay(100); // Give some time for the process to be killed
    assertEquals(manager["processes"].get(pid)?.status, "stopped", "Process should be stopped after killing");
  });

  await t.step("should monitor process resources", async () => {
    const manager = new EnhancedProcessManager();
    const pid = await manager.startProcess("yes > /dev/null");
    await delay(1000); // Give some time for the process to run and be monitored
    const processInfo = manager["processes"].get(pid);
    assertExists(processInfo);
    assertEquals(typeof processInfo.cpuUsage, "number");
    assertEquals(typeof processInfo.memoryUsage, "number");
    await manager.killProcess(pid);
  });

  await t.step("should handle process output", async () => {
    const manager = new EnhancedProcessManager();
    let outputReceived = false;
    manager["broadcastProcessOutput"] = (pid: number, output: string) => {
      assertEquals(typeof pid, "number");
      assertEquals(typeof output, "string");
      assertEquals(output.trim(), "hello");
      outputReceived = true;
    };
    await manager.startProcess("echo hello");
    await delay(100); // Give some time for the process to run and output to be processed
    assertEquals(outputReceived, true, "Process output should have been received");
  });

  await t.step("should get system memory info", async () => {
    const manager = new EnhancedProcessManager();
    const memInfo = await manager.getSystemMemoryInfo();
    assertExists(memInfo);
    assertEquals(typeof memInfo.total, "number");
    assertEquals(typeof memInfo.free, "number");
    assertEquals(typeof memInfo.available, "number");
  });

  await t.step("should handle WebSocket connections", async () => {
    const manager = new EnhancedProcessManager();
    let updateReceived = false;
    const mockWebSocket = {
      send: (message: string) => {
        const data = JSON.parse(message);
        assertEquals(data.type, "processUpdate");
        assertExists(data.processes);
        updateReceived = true;
      },
      on: () => {},
    };
    manager["handleWebSocketConnection"](mockWebSocket as any);
    assertEquals(updateReceived, true, "WebSocket client should receive an update");
  });
});

TypeError: Unknown export './server' for '@std/http@1.0.7'.
  Package exports:
 * .
 * ./cookie
 * ./etag
 * ./file-server
 * ./unstable-header
 * ./unstable-method
 * ./negotiation
 * ./server-sent-event-stream
 * ./status
 * ./unstable-signed-cookie
 * ./user-agent
 * ./unstable-route